# Metric (Lasot and vot)

## 1.1 Success and Precsion

In [18]:
import math
import numpy as np
from typing import List, Iterable, Tuple, Any
import itertools
import sys
sys.path.append('/home/lz/anaconda3/envs/nbconda/lib/python3.9/site-packages/vot/')
from attributee import Float, Integer, Boolean, Include
import os
from vot.tracker import Tracker
from vot.dataset import Sequence
from vot.region import Region, RegionType, calculate_overlaps
from vot.experiment import Experiment
from vot.experiment.multirun import UnsupervisedExperiment
from vot.analysis import SequenceAggregator, Analysis, SeparableAnalysis, \
    MissingResultsException, Measure, Sorting, Curve, Plot, SequenceAggregator, \
    Axes, analysis_registry
from vot.utilities.data import Grid
import numpy as np
from shapely.geometry import Polygon

import logging
log_path = os.path.join(os.getcwd(), 'metric.log')
logging.basicConfig(filename=log_path, level=logging.DEBUG, filemode='w', format='%(levelname)s:%(asctime)s:%(message)s', datefmt='%Y-%d-%m %H:%M:%S')

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count




def estimateIOU(box_a, box_b):
    if str(box_b[0]) == 'nan':
        return 0
    areaA = box_a[2] * box_a[3]
    areaB = box_b[2] * box_b[3]

    area_sum = areaA + areaB
    max_a_x = box_a[0] + box_a[2]
    max_a_y = box_a[1] + box_a[3]

    max_b_x = box_b[0] + box_b[2]
    max_b_y = box_b[1] + box_b[3]

    inter_x_max = min(max_a_x, max_b_x)
    inter_y_max = min(max_a_y, max_b_y)
    inter_x_min = max(box_a[0], box_b[0])
    inter_y_min = max(box_a[1], box_b[1])
    inter_w = inter_x_max - inter_x_min
    inter_h = inter_y_max - inter_y_min
    if inter_w<=0 or inter_h<=0:
        inter_area = 0 
    else:
        inter_area = inter_w * inter_h

    overlap = inter_area  / (area_sum - inter_area)
    return overlap





class Success(object):
    """Computes and stores the Success"""

    def __init__(self, n=21, max_overlap=1):
        self.max_overlap = max_overlap
        self.Xaxis = np.linspace(0, self.max_overlap, n)
        self.reset()

    def reset(self):
        self.overlaps = []

    def add_overlap(self, val):
        self.overlaps.append(val)

    @property
    def count(self):
        return len(self.overlaps)

    @property
    def value(self):
        # succ = [
        #     np.sum(i >= thres
        #            for i in self.overlaps).astype(float) / self.count
        #     for thres in self.Xaxis
        # ]
        succ = [
            np.sum(np.fromiter((i >= thres
                   for i in self.overlaps), dtype=float)) / self.count
            for thres in self.Xaxis
        ]
        
    
        return np.array(succ)

    @property
    def average(self):
        if len(self.overlaps) == 0:
            return 0
        return np.trapz(self.value, x=self.Xaxis) * 100 / self.max_overlap


class Sequence_t(object):

    def __init__(self, name: str, dataset='/data1/yjy/rgbd_benchmark/alldata/'):
        self._name = name
        self._dataset = dataset
        self._path = self._dataset + self._name
        self._numframe = self.num_frame

    @property
    def num_frame(self):
        seq_list = os.listdir(os.path.join(self._path, 'color'))
        return len(seq_list)
    @property
    def name(self) -> str:
        return self._name

    @property
    def identifier(self) -> str:
        return self._name

    @property
    def dataset(self):
        return self._dataset

    @property
    def gt(self):
        gtfile = os.path.join(self._path, 'groundtruth.txt')
        with open(gtfile, 'r') as f:
            value = np.loadtxt(f, delimiter=',')
        
        return value 
    @property
    def invisible(self):
        full_occlusion = os.path.join(self._path, 'full-occlusion.tag')
        if not os.path.exists(full_occlusion):
            value = np.array([0 for i in range(self._numframe)])
        else:
            with open(full_occlusion, 'r') as f:
                value = np.loadtxt(f)
        return value

class Tracking(object):
    def __init__(self, name: str, path='/data1/yjy/rgbd_benchmark/all_benchmark/results'):
        self._name = name
        self._path = os.path.join(path, self._name) 
        self._prre = PrRe()
        self._seqlist = self.re_list()
        self._numseq = len(self._seqlist)
        self._lackseq = [] 
        
    @property
    def name(self) -> str:
        return self._name + '_' + self._path.split('/')[-3]
    @property
    def prre(self):
        return self._prre
    def lack(self, seq):
        self._lackseq.append(seq)
    def re_list(self):
        seq_list = []
        all_list = os.listdir(os.path.join(self._path, 'rgbd-unsupervised'))
        # for name in all_list:
        #     if '_001.txt' in name:
        #         seq = name.split('_001.txt')[0]
        #         seq_list.append(seq)
        return all_list

    def prebox_conf(self, sequence):
        sequence_dir = os.path.join(self._path, 'rgbd-unsupervised', sequence)
        boxtxt = os.path.join(sequence_dir, '{}_001.txt'.format(sequence))
        try:
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        except:
            logging.debug('use \ t in {}'.format(self._name))
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter='\t', skiprows=1)
        conftxt = os.path.join(sequence_dir, '{}_001_confidence.value'.format(sequence))
        if not os.path.exists(conftxt):
            conftxt = os.path.join(sequence_dir, '{}_001_confidence.txt'.format(sequence))
        
        if not os.path.exists(conftxt):
           confidence = [1 for i in range(len(pre_value))]
        else:
            with open(conftxt, 'r') as f:
                confidence = np.loadtxt(f, skiprows=1)
        return pre_value, confidence


    









## 1.2 precision and recall

In [19]:
class PrRe(object):
    """Computes and stores the Success"""

    def __init__(self):
        self.reset()
        self.thresholds = np.linspace(1, 0, 100)
    def reset(self):
        self.overlaps = []
        self.confidence = []
        self.visible = []

    def add_overlap(self, val):
        self.overlaps.append(val)

    def add_list_iou(self, overlap:list):
        self.overlaps = np.concatenate((self.overlaps, overlap))
    
    def add_confidence(self, confidence:list):
        self.confidence = np.concatenate((self.confidence, confidence))
    def add_visible(self, visible:list):
        self.visible = np.concatenate((self.visible, visible))

    @property
    def count(self):
        return len(self.overlaps)

    @property
    def value(self):

        # succ = [
        #     np.sum(np.fromiter((i >= thres
        #            for i in self.overlaps), dtype=float)) / self.count
        #     for thres in self.Xaxis
        # ]
        # return np.array(succ)
        
        n_visible = len([vis for vis in self.visible if vis == True])
        precision = len(self.thresholds) * [float(0)]
        recall = len(self.thresholds) * [float(0)]

        for i, threshold in enumerate(self.thresholds):

            subset = self.confidence >= threshold
            
            if np.sum(subset) == 0:
                precision[i] = 1
                recall[i] = 0
            else:
                try:
                    # precision[i] = np.mean(self.overlaps[subset])
                    precision[i] = np.mean(self.overlaps[subset])
                    recall[i] = np.sum(self.overlaps[subset]) / n_visible
                except:
                    print('exception')
                if precision[i] == np.nan or recall[i] == np.nan:
                    print('nan')
        return precision, recall

    @property
    def fscore(self):
        pr, re = self.value

        pr_score = abs(np.trapz(pr, self.thresholds))
        re_score = abs(np.trapz(re, self.thresholds))
        # pr_score = np.sum(pr)/100
        # re_score = np.sum(re)/100
        fscore = 2*pr_score*re_score/(pr_score+re_score)
        return pr_score, re_score, fscore
        
class Recall(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def compute_tpr_curves(trajectory: Tracking, sequence: Sequence_t, all_prre: PrRe):
    
    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.prebox_conf(sequence.name)
    gt = sequence.gt 

    # firstframe in each sequence
    overlaps = np.concatenate(([1], np.array([estimateIOU(prebbox[i], gt[i+1] ) for i in range(len(prebbox))])))

    confidence = np.concatenate(([1], np.array(confidence)))


    #n_visible = len([region for region in sequence.groundtruth() if region.type is not RegionType.SPECIAL])
    # sequence.invisible (full-occlusion tag) if invisible= 1 full-occlusion invisible
    visible = np.array(sequence.invisible) < 1
    visible = visible + 0
    try:
        assert len(overlaps) == len(visible) == len(confidence)
    except:
        print("assert not equal")    
    all_prre.add_list_iou(overlaps)
    all_prre.add_visible(visible)
    all_prre.add_confidence(confidence) 


### 1 analysis of temporal

In [ ]:
normal_results = '/data1/yjy/rgbd_benchmark/all_benchmark/normal/results'
seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
sre_workspace = '/data1/yjy/rgbd_benchmark/all_benchmark/SRE_workspace/'
all_robust = [os.path.join(sre_workspace,sre) for sre in os.listdir(sre_workspace)]
all_trackers = []
origin_trackers = [Tracking(tracker, path=normal_results) for tracker in normal_results] 
all_trackers.extend(origin_trackers)
for robust in all_robust:
    trackers = [Tracking(tracker, path=os.path.join(robust, 'results')) for tracker in os.listdir(os.path.join(robust, 'results'))] 
    all_trackers.extend(trackers)
# all_trackers = [Tracking(tracker,path=os.path.join()) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/SRE_workspace/')]
all_sequence = [Sequence_t(seq) for seq in seq_list]

for trackers in all_trackers:
    print(trackers.name)
    if not trackers._name == 'TSDM':
        continue 
    for sequence in all_sequence:
        
        if sequence.name in trackers._seqlist:
            compute_tpr_curves(trackers, sequence, trackers._prre)
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            trackers.lack(sequence.name)
            continue
    
    pr,re,fscore = trackers._prre.fscore
    print('Trackers: {}  Seq_num: {} frame_num: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, trackers._numseq, trackers._prre.count, pr, re, fscore))
    logging.info('Trackers: {}  Seq_num: {} frame_num: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, trackers._numseq, trackers._prre.count, pr, re, fscore))


### 2 read full-occlusion tag 

In [20]:


normal_results = '/data1/yjy/rgbd_benchmark/all_benchmark/normal/results'
seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
sre_workspace = '/data1/yjy/rgbd_benchmark/all_benchmark/SRE_workspace/'
all_robust = [os.path.join(sre_workspace,sre) for sre in os.listdir(sre_workspace)]
all_trackers = []
origin_trackers = [Tracking(tracker, path=normal_results) for tracker in normal_results] 
all_trackers.extend(origin_trackers)
for robust in all_robust:
    trackers = [Tracking(tracker, path=os.path.join(robust, 'results')) for tracker in os.listdir(os.path.join(robust, 'results'))] 
    all_trackers.extend(trackers)
# all_trackers = [Tracking(tracker,path=os.path.join()) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/SRE_workspace/')]
all_sequence = [Sequence_t(seq) for seq in seq_list]

for trackers in all_trackers:
    print(trackers.name)
    if not trackers._name == 'TSDM':
        continue 
    for sequence in all_sequence:
        
        if sequence.name in trackers._seqlist:
            compute_tpr_curves(trackers, sequence, trackers._prre)
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            trackers.lack(sequence.name)
            continue
    
    pr,re,fscore = trackers._prre.fscore
    print('Trackers: {}  Seq_num: {} frame_num: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, trackers._numseq, trackers._prre.count, pr, re, fscore))
    logging.info('Trackers: {}  Seq_num: {} frame_num: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, trackers._numseq, trackers._prre.count, pr, re, fscore))


        
    
    
    



    
    

Siam_LTD_sre1
TSDM_sre1
Trackers: TSDM_sre1  Seq_num: 171 frame_num: 188710  pr: 0.4646910323075539  re: 0.43620599199470506  fscore: 0.44999818458885726
DDiMP_sre1
TSDM_sre4
Trackers: TSDM_sre4  Seq_num: 171 frame_num: 188710  pr: 0.463644771891277  re: 0.4365822121522031  fscore: 0.44970671564610787
DDiMP_sre4
TSDM_sre2
Trackers: TSDM_sre2  Seq_num: 171 frame_num: 188710  pr: 0.47442429918225754  re: 0.44926423582489805  fscore: 0.4615016039518745
DDiMP_sre2
Siam_LTD_sre3
TSDM_sre3
Trackers: TSDM_sre3  Seq_num: 171 frame_num: 188710  pr: 0.4612715525879103  re: 0.42620253364772964  fscore: 0.44304415748403025
TSDM_sre6
Trackers: TSDM_sre6  Seq_num: 171 frame_num: 188710  pr: 0.444246149177708  re: 0.4210489389419731  fscore: 0.4323366035667597
DDiMP_sre6
Siam_LTD_sre5
TSDM_sre5
Trackers: TSDM_sre5  Seq_num: 171 frame_num: 188710  pr: 0.4416310165831166  re: 0.41227174581113146  fscore: 0.4264466593375496
DDiMP_sre5
TSDM_sre7
Trackers: TSDM_sre7  Seq_num: 171 frame_num: 188710  pr: 0.

In [121]:
print('name: {}, frame: {}'.format(all_sequence[1].name, all_sequence[1]._numframe))
# prebbox, confidence = all_trackers[2].prebox_conf(all_sequence[2].name)
# for trackers in all_trackers:

#     print('name: {} \t \t number of seq {} '.format(trackers.name, trackers._numseq))     

name: container_room_noocc_1, frame: 1644


#### 2 read confidence score and tracking result (s)

## calculate auc of iou 

In [30]:
import os
# tracker_name = ['DDiMP', 'ATCAIS', 'DRefine', 'SLMD', 'Siam_LTD', 'TSDM', 'iiau_rgbd']
tracker_name = ['ATCAIS19']
gt_dir = '/data1/yjy/rgbd_benchmark/stc_benchmark/RGBDdataset/'
su = Success()
for tracker in tracker_name:
    su.reset()
    tracking_result_dir = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/stc_workspace/results/{}/rgbd-unsupervised'.format(tracker)
    seqlist = os.listdir(tracking_result_dir)
    for seq in seqlist:
        gt_seq = os.path.join(gt_dir, seq, 'groundtruth.txt' )
        pre_seq = os.path.join(tracking_result_dir, '{}/{}_001.txt'.format(seq, seq))
        with open(gt_seq, 'r') as f:
            gt_value = np.loadtxt(f, delimiter=',', skiprows=1)
        with open(pre_seq, 'r') as f:
            pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        if not len(gt_value) == len(pre_value):
            raise TypeError
        for i in range(len(gt_value)):
            iou = estimateIOU(gt_value[i], pre_value[i])
            su.add_overlap(iou)
    print('{} iou auc: {}'.format(tracker, su.average))
    


/tmp/ipykernel_35807/2209188759.py:76: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  np.sum(i >= thres


ATCAIS19 iou auc: 66.43529551954242


### stc tracker

In [32]:
import os
# tracker_name = ['DDiMP', 'ATCAIS', 'DRefine', 'SLMD', 'Siam_LTD', 'TSDM', 'iiau_rgbd']
tracker_name = ['stc']
gt_dir = '/data1/yjy/rgbd_benchmark/stc_benchmark/RGBDdataset/'
su = Success()
for tracker in tracker_name:
    su.reset()
    tracking_result_dir = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/stc_workspace/results/{}/rgbd-unsupervised'.format(tracker)
    seqlist = os.listdir(tracking_result_dir)
    for seqtxt in seqlist:
        gt_seq = os.path.join(gt_dir, seqtxt.split('.')[0], 'groundtruth.txt' )
        pre_seq = os.path.join(tracking_result_dir, seqtxt)
        with open(gt_seq, 'r') as f:
            gt_value = np.loadtxt(f, delimiter=',', skiprows=1)
        with open(pre_seq, 'r') as f:
            pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        if not len(gt_value) == len(pre_value):
            raise TypeError
        for i in range(len(gt_value)):
            iou = estimateIOU(gt_value[i], pre_value[i])
            su.add_overlap(iou)
    print('{} iou auc: {}'.format(tracker, su.average))

stc iou auc: 39.79200786112021


### copy result to all_benchmark